In [1]:
import pandas as pd
import dgl
import dgl.nn
import os
import numpy as np
import torch

Using backend: pytorch


In [4]:
data_dir = '/home/ZYH/ISM/deezer_europe/'
edges_path = os.path.join(data_dir, 'deezer_europe_edges.csv')
nodes_feature_path = os.path.join(data_dir, 'deezer_europe_features.txt')
nodes_label_path = os.path.join(data_dir, 'deezer_europe_target.csv')
f = open(nodes_feature_path, "r")
node_feature_dict = dict(eval(f.read()))
edges = pd.read_csv(edges_path)
sources_node = np.array(edges['node_1'])
dist_nodes = np.array(edges['node_2'])
nodes_label = pd.read_csv(nodes_label_path)
# create graph
num_nodes = len(nodes_label)
g = dgl.DGLGraph((sources_node, dist_nodes), num_nodes=num_nodes)
feature_len_list = []
for key,value in node_feature_dict.items():
    feature_len_list.append(len(value))
feature_list = np.zeros((num_nodes, max(feature_len_list)))
for i, (key,value) in enumerate(node_feature_dict.items()):
    feature_list[i][:len(value)] = np.array(value)
g.ndata['feat'] = torch.tensor(feature_list).float()
g.ndata['label'] = torch.tensor(np.array(nodes_label['target']))
g.ndata['train_mask'] = torch.cat((torch.tensor([True]*18000), torch.tensor([False]*10281)),dim=-1)
g.ndata['val_mask'] = torch.cat((torch.tensor([False]*18000), torch.tensor([True]*5000),torch.tensor([False]*5281)),dim=-1)
g.ndata['test_mask'] = torch.cat((torch.tensor([False]*23000), torch.tensor([True]*5281)),dim=-1)
g = dgl.add_self_loop(g)


[0 0 0 ... 0 1 1]


In [3]:
g


Graph(num_nodes=28281, num_edges=121033,
      ndata_schemes={'feat': Scheme(shape=(1715,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [5]:
from dgl.nn import GraphConv
import torch.nn as nn

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 1024, 2)

In [7]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(500):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 0, loss: 2407.105, val acc: 0.010 (best 0.010), test acc: 0.011 (best 0.011)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 5, loss: 227.917, val acc: 0.236 (best 0.380), test acc: 0.211 (best 0.394)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 10, loss: 102.261, val acc: 0.538 (best 0.538), test acc: 0.537 (best 0.537)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 15, loss: 56.470, val acc: 0.515 (best 0.538), test acc: 0.517 (best 0.537)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1]

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 170, loss: 0.878, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 175, loss: 0.869, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 180, loss: 0.860, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 185, loss: 0.853, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 335, loss: 0.758, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 340, loss: 0.768, val acc: 0.492 (best 0.560), test acc: 0.491 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 345, loss: 0.757, val acc: 0.558 (best 0.560), test acc: 0.550 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 350, loss: 0.762, val acc: 0.480 (best 0.560), test acc: 0.490 (best 0.547)


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 500, loss: 0.732, val acc: 0.507 (best 0.560), test acc: 0.497 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 505, loss: 0.733, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 510, loss: 0.734, val acc: 0.487 (best 0.560), test acc: 0.494 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 515, loss: 0.734, val acc: 0.559 (best 0.560), test acc: 0.549 (best 0.547)


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 665, loss: 0.724, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 670, loss: 0.725, val acc: 0.486 (best 0.560), test acc: 0.496 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 675, loss: 0.722, val acc: 0.558 (best 0.560), test acc: 0.549 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 680, loss: 0.719, val acc: 0.512 (best 0.560), test acc: 0.495 (best 0.547)


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 830, loss: 0.713, val acc: 0.510 (best 0.560), test acc: 0.504 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 835, loss: 0.710, val acc: 0.557 (best 0.560), test acc: 0.548 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 840, loss: 0.709, val acc: 0.534 (best 0.560), test acc: 0.506 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 845, loss: 0.714, val acc: 0.557 (best 0.560), test acc: 0.551 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 995, loss: 0.705, val acc: 0.557 (best 0.560), test acc: 0.547 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])


In [8]:
from dgl.nn import TAGConv
import torch.nn as nn

class TAG(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(TAG, self).__init__()
        self.conv1 = TAGConv(in_feats, h_feats)
        self.conv2 = TAGConv(h_feats, num_classes)
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = TAG(g.ndata['feat'].shape[1], 1024, 18)

In [9]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(500):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = TAG(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 0, loss: 7635.862, val acc: 0.009 (best 0.009), test acc: 0.009 (best 0.009)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 5, loss: 3698.665, val acc: 0.546 (best 0.554), test acc: 0.531 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 10, loss: 1400.571, val acc: 0.466 (best 0.554), test acc: 0.484 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 15, loss: 1118.100, val acc: 0.530 (best 0.554), test acc: 0.525 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 165, loss: 23.015, val acc: 0.509 (best 0.554), test acc: 0.515 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 170, loss: 18.720, val acc: 0.506 (best 0.554), test acc: 0.505 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 175, loss: 19.578, val acc: 0.492 (best 0.554), test acc: 0.497 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 180, loss: 21.257, val acc: 0.512 (best 0.554), test acc: 0.519 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 330, loss: 0.950, val acc: 0.530 (best 0.554), test acc: 0.524 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 335, loss: 0.924, val acc: 0.532 (best 0.554), test acc: 0.525 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 340, loss: 0.903, val acc: 0.537 (best 0.554), test acc: 0.533 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 345, loss: 0.884, val acc: 0.541 (best 0.554), test acc: 0.531 (best 0.536)


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 495, loss: 0.741, val acc: 0.543 (best 0.554), test acc: 0.535 (best 0.536)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])


In [10]:
from dgl.nn import EdgeConv
import torch.nn as nn

class EDGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(EDGE, self).__init__()
        self.conv1 = EdgeConv(in_feats, h_feats)
        self.conv2 = EdgeConv(h_feats, num_classes)
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = EDGE(g.ndata['feat'].shape[1], 1024, 18)

In [12]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(500):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = EDGE(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 0, loss: 1698.709, val acc: 0.036 (best 0.036), test acc: 0.031 (best 0.031)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 5, loss: 1365.003, val acc: 0.444 (best 0.444), test acc: 0.447 (best 0.447)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 10, loss: 783.639, val acc: 0.465 (best 0.518), test acc: 0.459 (best 0.504)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 15, loss: 419.591, val acc: 0.498 (best 0.518), test acc: 0.503 (best 0.504)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 165, loss: 37.622, val acc: 0.480 (best 0.556), test acc: 0.486 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 170, loss: 26.734, val acc: 0.472 (best 0.556), test acc: 0.467 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 175, loss: 14.499, val acc: 0.548 (best 0.556), test acc: 0.542 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 180, loss: 8.740, val acc: 0.542 (best 0.556), test acc: 0.543 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 330, loss: 1.882, val acc: 0.521 (best 0.556), test acc: 0.517 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 335, loss: 2.886, val acc: 0.545 (best 0.556), test acc: 0.544 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 340, loss: 3.385, val acc: 0.489 (best 0.556), test acc: 0.476 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 345, loss: 1.675, val acc: 0.535 (best 0.556), test acc: 0.531 (best 0.543)


tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 495, loss: 0.792, val acc: 0.535 (best 0.556), test acc: 0.533 (best 0.543)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])


In [13]:
from dgl.nn import SAGEConv
import torch.nn as nn

class SAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(SAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,'pool')
        self.conv2 = SAGEConv(h_feats, num_classes,'pool')
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = SAGE(g.ndata['feat'].shape[1], 1024, 18)

In [14]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(500):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = SAGE(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 0, loss: 10818.516, val acc: 0.077 (best 0.077), test acc: 0.076 (best 0.076)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 5, loss: 48669.340, val acc: 0.511 (best 0.548), test acc: 0.485 (best 0.549)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 10, loss: 19245.795, val acc: 0.484 (best 0.555), test acc: 0.492 (best 0.549)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 15, loss: 14808.054, val acc: 0.517 (best 0.555), test acc: 0.500 (best 0.549)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 165, loss: 233.163, val acc: 0.551 (best 0.560), test acc: 0.549 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 170, loss: 180.491, val acc: 0.497 (best 0.560), test acc: 0.498 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 175, loss: 172.512, val acc: 0.481 (best 0.560), test acc: 0.497 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 180, loss: 152.347, val acc: 0.495 (best 0.560), test acc: 0.492 (best 0.547)
tensor([0, 0, 0,  ..., 1,

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 330, loss: 44.111, val acc: 0.528 (best 0.560), test acc: 0.529 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 335, loss: 22.727, val acc: 0.525 (best 0.560), test acc: 0.530 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 340, loss: 69.283, val acc: 0.512 (best 0.560), test acc: 0.505 (best 0.547)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 345, loss: 162.343, val acc: 0.523 (best 0.560), test acc: 0.520 (best 0.

RuntimeError: 

In [15]:
from dgl.nn import AGNNConv
import torch.nn as nn

class AGNN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(AGNN, self).__init__()
        self.conv1 = AGNNConv()
        self.conv2 = AGNNConv()

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = AGNN(g,g.ndata['feat'].shape[1],18)

In [16]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(500):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = AGNN(g,g.ndata['feat'].shape[1], 18)
train(g, model)

tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 0, loss: 14149.051, val acc: 0.012 (best 0.012), test acc: 0.010 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 5, loss: 14097.883, val acc: 0.011 (best 0.012), test acc: 0.010 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 10, loss: 14050.402, val acc: 0.011 (best 0.012), test acc: 0.010 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 15, loss: 14006.669, val acc: 0.011 (best 0.012), test acc: 0.010 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 165, loss: 13325.232, val acc: 0.011 (best 0.012), test acc: 0.012 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 170, loss: 13312.457, val acc: 0.011 (best 0.012), test acc: 0.012 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 175, loss: 13300.057, val acc: 0.011 (best 0.012), test acc: 0.012 (best 0.010)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 180, loss: 13288.018, val acc: 0.011 (best 0.012), test acc: 0.0

tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 325, loss: 13125.410, val acc: 0.014 (best 0.014), test acc: 0.013 (best 0.013)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 330, loss: 13124.315, val acc: 0.014 (best 0.014), test acc: 0.013 (best 0.013)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 335, loss: 13123.248, val acc: 0.014 (best 0.014), test acc: 0.013 (best 0.013)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 340, loss: 13122.259, val acc: 0.014 (best 0.014), test acc: 0.013 (best 0.013)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0, 

tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 485, loss: 13113.047, val acc: 0.015 (best 0.015), test acc: 0.014 (best 0.014)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 490, loss: 13112.976, val acc: 0.015 (best 0.015), test acc: 0.014 (best 0.014)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
In epoch 495, loss: 13112.887, val acc: 0.015 (best 0.015), test acc: 0.014 (best 0.014)
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
tensor([0, 0, 0,  ..., 1, 1, 1])
